Extracting Deep Learning Features from Images

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable

from IPython.display import Image
from IPython.core.display import HTML

from PIL import Image
import requests
from io import BytesIO

In [ ]:
# Resnet loading

model = models.resnet50(pretrained='imagenet')

In [ ]:
#Resize the image to 224x224 px
# input size도 변경(resnet 모델 변경시)

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]) # scaling은 대부분 이 사이즈로 실행
normalize = transforms.Normalize((0.5,),(0.5,))

to_tensor = transforms.ToTensor()

# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

def extract_feature_vector(img):
    # 2. Create a PyTorch Variable with the transformed image
    #Unsqueeze actually converts to a tensor by adding the number of images as another dimension.
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))

    # 3. Create a vector of zeros that will hold our feature vector
    # The 'avgpool' layer has an output size of 512 -> output size로 변경해야함
    my_embedding = torch.zeros(1, 2048, 1, 1)

    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)

    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)

    # 6. Run the model on our transformed image
    model(t_img)

    # 7. Detach our copy function from the layer
    h.remove()

    # 8. Return the feature vector
    return my_embedding.squeeze().numpy()

In [ ]:
## code test 

image = Image.open("sample/20070@N98/173632.jpg")
a = extract_feature_vector(image)
a

load image files & data prerprocessing

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook

In [ ]:
data_s = pd.read_csv('data_s.csv')
data_s = data_s.dropna(axis = 0)

# 현재 경로 추가
data_s['img_path_2'] = 'your path' + data_s['path']
data_s.info()

In [ ]:
img_list_jpg = data_s['img_path_2']

list = []

m = 0
for i in tqdm_notebook(img_list_jpg):
    try:
      image = Image.open(i)
      feature = extract_feature_vector(image)
      list.append(feature)
      print(m)
      m+=1

    except:
      list.append(np.nan) # 예외 발생시 nan 처리
      print(m,'error')
      m+=1

# 원본 데이터 저장
df_resnet_features = pd.DataFrame(list)
df_resnet_features.to_csv('df_resnet_features.csv')
